# Asteroid Classification

Predicting whether an asteroid is hazardous or not.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
    

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#load data
asteroids_data = pd.read_csv('/kaggle/input/nasa-asteroids-classification/nasa.csv')

In [ ]:
asteroids_data.head()

The target variable, **Hazardous**, has labels True and False. We will transform these labels to the binary, 1 and 0, where 1 is True and 0 is False.

In [ ]:
#binarising Hazardous
asteroids_data['Hazardous'].replace({True: 1, False: 0}, inplace = True)

In [ ]:
asteroids_data['Hazardous'].value_counts()


**ATTRIBUTES**
* **Neo Reference ID** - Near Earth Object (NEO) reference ID number for an asteroid (or a comet) which comes close to earth
* **Name** - 'Name' of asteroid (same as NEO Reference ID)
* **Absolute Magnitude** - A measure of the asteroid's luminosity (in H) (the brightness of an asteroid if it is 1 astronomical unit away from both the Sun and the observer, and the angle between the Sun, asteroid, and Earth is 0 degrees) 
* **Est Dia in (in KM, M, Miles, and Feet) (min)** - Minimum estimated diameter of the asteroid (Note: Since asteroids cannot be directly measured and because they have irregular shapes, their diameters are estimates. These estimates are calculated using its absolute magnitude and geometric albedo.)   
* **Est Dia in (in KM, M, Miles, and Feet) (max)**  - Maximum estimated diameter of the asteroid      
* **Close Approach Date** - Date at which the asteroid approaches close to Earth      
* **Epoch Date Close Approach** - Date at which the asteroid approaches close to Earth (in epoch time)   
* **Relative Velocity (in km per sec, km per hr, and miles per hour)** - Asteroid's velocity relative to earth
* **Miss Dist.(in Astronomical, lunar, km, and miles)** - Distance by which the asteroid misses Earth
* **Orbiting Body**                   
* **Orbit ID** - An ID of JPL NEA orbit that JPL Nasa uses in its analysis               
* **Orbit Determination Date** - Date at which the asteroid's orbit was determined
* **Orbit Uncertainity** - A measure of the uncertainity ('measurement errors') in the calculated orbit 
* **Minimum Orbit Intersection** - The closest distance between Earth and the asteroid in their respective orbits (in astronomical units)
* **Jupiter Tisserand Invariant** -  A value used to differentiate between asteroids and Jupiter-family comets    
* **Epoch Osculation** - The instance of time at which the asteroid's position and velocity vectors (from which its osculating orbit is calculated) is specified           
* **Eccentricity** - A value which specifies by how much the asteroid's orbit deviates from a perfect circle               
* **Semi Major Axis** - The longest radius of an elliptical orbit; a easure of the asteroid's average distance from the Sun  (asteroids orbit the Sun)              
* **Inclination** - Measures the tilt of the asteroid's orbit around the Sun                 
* **Asc Node Longitude** - (copying from NASA) 'Angle in the ecliptic plane between the inertial-frame x-axis and the line through the ascending node'
* **Orbital Period** - Time taken for asteroid to complete a single orbit around the Sun
* **Perihelion Distance** - Distance of point in asteroid's orbit which is closest to the Sun
* **Perihelion Arg** - (copying from Nasa) 'The angle (in the body's orbit plane) between the ascending node line and perihelion measured in the direction of the body's orbit'
* **Aphelion Dist** - Distance of point in asteroid's orbit which is farthest from the Sun
* **Perihelion Time** - Length of time of asteroid's passage through the perihelion stage
* **Mean Anomaly** - (copying from Nasa) 'The product of an orbiting body's mean motion and time past perihelion passage'
* **Mean Motion** - (copying from Nasa) 'The angular speed required for a body to make one orbit around an ideal ellipse with a specific semi-major axis'
* **Equinox** - An astronomical standard to measure against (currently 'J2000.0')
* **Hazardous** - Is the asteroid hazardous? (True or False)  

First, we will collate all the variables (except the target **Hazardous**) in a list called **features**, and we will drop the features we have decided not to include in the model as we go along. 

In [ ]:
#list of features
features = asteroids_data.columns.tolist()
features = features[0:-1]
#check
print(features)

In [ ]:
asteroids_data.info()

This is a small dataset with only 4687 instances. There are 40 attributes but a few of them measure the same feature of the asteroid but in different standards. These attributes are **Est Dia** (in various measures), **Close Approach Date** and **Epoch Date Close Approach**, **Relative Velocity** (in various measures), and **Miss Dist.**(in various measures). There are no missing values.

There are 5 non-numerical attribute, of which one is a boolean (**Hazardous**), and the others (**Close Approach Date**, **Orbiting Body**,**Orbit Determination Date**, **Equinox**) are of type object. Let's have a look at the object type attribute.

In [ ]:
asteroids_data[['Close Approach Date', 'Orbiting Body','Orbit Determination Date', 'Equinox']].head()

Two of the object types are dates.  We do not need the date features for our model. 

How many unique vales do **Orbiting Body** and **Equinox** have?

In [ ]:
#unique values
print('Orbiting Body unique values: {}'.format(asteroids_data['Orbiting Body'].unique()))
print('Equinox unique values: {}'.format(asteroids_data['Equinox'].unique()))

Each of the two features have only a single unique value so they are useless for training the model.

Let's examine the numerical type values.

* **NEO Reference ID, Name, and Orbit ID** are variables which are used by researchers to identify the asteroid or an orbit. We do not need these for the model.
* Among the featuress which measure the same thing in different standards, we will select **Est Dia in KM(min), Est Dia in KM(max), Relative Velocity in km per hour**, and **Miss Dist.(kilometers)**. We will drop the other representations of these measurements.
* Many of the reamining attributes of numeric type are orbital elements which are used to calculate the asteroid's orbit.


Before we move on, we will first drop the features we do not need for the model from out list of features.

In [ ]:
for f in ['Neo Reference ID', 'Name', 'Est Dia in M(min)', 'Est Dia in M(max)', 'Est Dia in Miles(min)', 'Est Dia in Miles(max)', 
          'Est Dia in Feet(min)', 'Est Dia in Feet(max)', 'Close Approach Date', 'Epoch Date Close Approach', 'Relative Velocity km per sec', 
          'Miles per hour', 'Miss Dist.(Astronomical)', 'Miss Dist.(lunar)', 'Miss Dist.(miles)', 'Orbiting Body', 'Orbit ID', 'Orbit Determination Date', 'Epoch Osculation', 'Equinox']:
    features.remove(f)

In [ ]:
print('There are now {} features.'.format(len(features)))

**CREATE TRAINING, VALIDATION, AND TEST SET**

We will now split the dataset into train, validation, and test sets in the proportions 60%, 20%, and 20% respectively.

In [ ]:
from sklearn.model_selection import train_test_split 

#split data into train, cross validation, and test sets
asteroid_set, asteroid_test = train_test_split(asteroids_data, test_size=0.2, random_state=7)
asteroid_train, asteroid_cv = train_test_split(asteroid_set, test_size=0.25, random_state=5)

NASA'a CNEOS defines[ petentially hazardous asteroids](https://cneos.jpl.nasa.gov/about/neo_groups.html) as an asteroid with "a minimum orbit intersection distance (MOID) of 0.05 au or less and an absolute magnitude (H) of 22.0 or less". Note: 1 au (astronomical unit) = 149597870700 m (roughly distance between Earth and Sun)

We will plot Minimum Orbit Intersection, Absolute Magnitude, and Hazardous in a scatter plot to see this relationship.

In [ ]:
#pairplot to see relationship between 'Minimum Orbit Intersection', 'Absolute Magnitude', and 'Hazardous'
sns.pairplot(asteroid_train[['Minimum Orbit Intersection', 'Absolute Magnitude', 'Hazardous' ]], diag_kind = 'hist', hue='Hazardous', palette = {1:'red', 0:'blue'})


In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(asteroid_train['Minimum Orbit Intersection'], asteroid_train['Absolute Magnitude'], hue = asteroid_train['Hazardous'], palette={1: 'red', 0:'blue'})
plt.show()

From these plots, we can see that asteroids in the datatset which are hazardous have an **Absolute Magnitude** between roughly 15 and 23 H, with a **Minimum Orbit Intersection** between roughly 0 and 0.05 au. This dataset reflects the NASA's definition of potentially hazardous asteroids. 

If we just have these two features to train a model to predict whether an asteroid is hazardous, we can implement a simple binary logistic regression classifier. Let's try that.

**BIVARIATE BINARY CLASSIFIER**

In [ ]:
asteroid_train[['Minimum Orbit Intersection','Absolute Magnitude']].describe()

From the plots and the table above, there are no significant outliers in the context of asteroid measurements. However, since a couple of points are scattered well away from the bulk of the distribution, we will use the robust scsler to scale the data before training. The robust scaler uses the interquartile range to scale, and is thus not influenced by outliers.

First, we will scale **Absolute Magnitude** and **Minimum Orbit Intersection** using sklearn's Robust scaler. We will scale these two features in the train, validation, and test sets.

In [ ]:
from sklearn.preprocessing import robust_scale

for col in ['Absolute Magnitude', 'Minimum Orbit Intersection']:
    asteroid_train[col + "_scaled"] = robust_scale(asteroid_train[col])
    asteroid_cv[col + "_scaled"] = robust_scale(asteroid_cv[col])
    asteroid_test[col + "_scaled"] = robust_scale(asteroid_test[col])
    features.append(col + '_scaled')
    features.remove(col)

We will now train a logistic regression model using these two scaled features on the training set, and test on the validation set.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

In [ ]:
lr = LogisticRegression()
lr.fit(asteroid_train[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']], asteroid_train['Hazardous'])
predictions = lr.predict(asteroid_cv[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']])

We know that the target variable skews to 0 (it has prominently more 0 labels than 1 labels), therefore accuracy alone will not be a good evaluation of the model. Below is a function that will return a confusion matrix and will calculate precision, recall, null accuracy, specificity, and the F_score.

In [ ]:
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from sklearn.metrics import accuracy_score


def evaluation(test_y, predictions):
    
    #accuracy score
    accuracy = accuracy_score(test_y, predictions)
    print("The classification accuracy is {:.2f} %." .format(accuracy*100))
    
  
    y_test_mean = test_y.mean()
    #null accuracy
    null_accuracy = max(y_test_mean, 1-y_test_mean)
    print('The null accuracy is {:.2f} %.'.format(null_accuracy*100))
    
    #confusion matrix
    skplt.metrics.plot_confusion_matrix(test_y, predictions)
    
    conf_matrix = confusion_matrix(test_y, predictions)
    
    TN = conf_matrix[0,0] #true negatives
    FP = conf_matrix[0,1] #false positives
    FN = conf_matrix[1,0] #false negatives
    TP = conf_matrix[1,1] #true positives
    
    #precision
    precision = TP/(TP+FP)*100
    print('The precision is {:.2f} %.'.format(precision))
    #sensitivity/ recall
    recall = TP/(FN+TP)*100
    print('The sensitivity/recall is {:.2f} %.'.format(recall))
    #specificity
    specificity = TN/(FP+TN)*100
    print('The specificity is {:.2f} %.'.format(specificity))
    #F_score
    F_score = (2*precision*recall)/(precision + recall)
    print('The F score is {:.2f} %.'.format(F_score))
    
    return None

In [ ]:
evaluation(asteroid_cv['Hazardous'], predictions)

Although the model predicts the label correctly 92.75% of the time, this does not give a true picture of the model's performance as the distribution of the target variable is skewed. This is reflected in the null accuracy score (the score if the model predicted the dominant class at all times). The classification accuracy is only four precentage points greater than the null accuracy. 

For the task of hazardous asteroid prediction, it is important that when an asteroid is hazardous, the model correctly predicts that it is. This is measured by recall and this model's recall is 89.91%. If we wanted to increase recall, we can lower the prediction threshold from 0.5 (the default in sklearn's Log Reg estimator) to a lower number. 


**SVM CLASSIFIER**

Could a SVM classifier yield better results? We will use the Gaussian kernel function. 

In [ ]:

from sklearn.svm import SVC

svm = SVC(C=1.5, kernel='rbf')
svm.fit(asteroid_train[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']], asteroid_train['Hazardous'])
svm_pred = svm.predict(asteroid_cv[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']])

In [ ]:
evaluation(asteroid_cv['Hazardous'], svm_pred)

The evaluation scores of the SVM classifier model on the validation data is an improvement on the evaluation scores of the Logistic Regression model. We will predict using the test set as well.

In [ ]:
#test set predictions
svm_pred = svm.predict(asteroid_test[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']])
evaluation(asteroid_test['Hazardous'], svm_pred)

The recall on the test set is higher than on the validation set, and the precision is lower.

**DECISION BOUNDARIES**

Both of the models above have only two features which means we can plot the decision boundary of each model. It will give a useful insight into how each classifier splits the data.

In [ ]:
X1 = asteroid_test['Absolute Magnitude_scaled']
X2 = asteroid_test['Minimum Orbit Intersection_scaled']
#create meshgrid for contour plot
x1, x2 = np.meshgrid(np.arange(X1.min() - 0.25, X1.max() + 0.25, 0.01), 
                     np.arange(X2.min() - 0.25, X2.max() + 0.25, 0.01))

**Decision Boundary for logistic classifier model**

In [ ]:
L = lr.predict(np.c_[x1.ravel(), x2.ravel()])
L = L.reshape(x1.shape)
#plot boundary and points
plt.figure(figsize=(12,10))
plt.contourf(x2, x1, L, cmap=plt.cm.twilight, alpha=0.8)
#plt.scatter(asteroid_test['Minimum Orbit Intersection_scaled'],asteroid_test['Absolute Magnitude_scaled'], 
          # cmap=plt.cm.coolwarm, s=20, edgecolors='k')
sns.scatterplot(asteroid_test['Minimum Orbit Intersection_scaled'],asteroid_test['Absolute Magnitude_scaled'], hue=asteroid_test['Hazardous'],  palette = {1:'red', 0:'blue'})
plt.show()

The decision boundary of the logistic regression classifier is a straight line that cuts through the points to minimise the mean squared error. The trained boundary captures most of the points of the test data set in the correct class but could be improved. Would polynomial features help?

**Decision Boundary for svm (with gaussian kernel) classifier model**

In [ ]:
Z = svm.predict(np.c_[x1.ravel(), x2.ravel()])
Z = Z.reshape(x1.shape)
#plot boundary and points
plt.figure(figsize=(12,10))
plt.contourf(x2, x1, Z, cmap=plt.cm.twilight, alpha=0.8)
#plt.scatter(asteroid_test['Minimum Orbit Intersection_scaled'],asteroid_test['Absolute Magnitude_scaled'], 
          # cmap=plt.cm.coolwarm, s=20, edgecolors='k')
sns.scatterplot(asteroid_test['Minimum Orbit Intersection_scaled'],asteroid_test['Absolute Magnitude_scaled'], hue=asteroid_test['Hazardous'],  palette = {1:'red', 0:'blue'})
plt.show()

The decision boundary of the svm classifier is a curved line that cuts through the points in a way as to maximise the distance between the positive and negative examples. Here the trained boundary curves around the test data points in such a way that it captures nearly all the points in its correct class.

Based on the our observation of the logistic regression model, it will be insightful to train and test a model with polynomial features of two or three degrees.

In [ ]:
#train and test Logistic regression model with polynomial features

from sklearn.preprocessing import PolynomialFeatures

for i in [2,3]:
    poly = PolynomialFeatures(i)
    X = poly.fit_transform(asteroid_train[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']])
    
    lr_poly = LogisticRegression()
    lr_poly.fit(X, asteroid_train['Hazardous'])
    X_cv = poly.fit_transform(asteroid_cv[['Absolute Magnitude_scaled','Minimum Orbit Intersection_scaled']])
    pred_poly = lr_poly.predict(X_cv)
    
    print('Logistic Regression with polynomial features degree {} scores: \n'.format(i))
    evaluation(asteroid_cv['Hazardous'], pred_poly)
    print('\n')


The logistic regression classifier with polynomial features has better precision, recall, and accuracy than the linear logistic regressor. There hardly any difference in the scores betwwen the model with polynomial features of degree 2 and that with polynomial features of degree 3. The svm model has better recall than any of the logistic regressor models. 